In [23]:
import os
import mlflow
from mlflow.pyfunc import PythonModel
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain import __version__ as langchain_version

class SousChefPreparation(PythonModel):
    def load_context(self, context):
        self.model_name = "llama3-70b-8192"
        self.api_key = os.getenv("GROQ_API_KEY")
        self.max_tokens = 1000

        self.llm = ChatGroq(model=self.model_name, api_key=self.api_key, max_tokens=self.max_tokens)

        template_instruction = (
            "Imagine you are a fine dining sous chef. Your task is to meticulously prepare for a dish, focusing on the mise-en-place process."
            "Given a recipe, your responsibilities are: "
            "1. List the Ingredients: Carefully itemize all ingredients required for the dish, ensuring every element is accounted for. "
            "2. Preparation Techniques: Describe the techniques and operations needed for preparing each ingredient. This includes cutting, "
            "processing, or any other form of preparation. Focus on the art of mise-en-place, ensuring everything is perfectly set up before cooking begins."
            "3. Ingredient Staging: Provide detailed instructions on how to stage and arrange each ingredient. Explain where each item should be placed for "
            "efficient access during the cooking process. Consider the timing and sequence of use for each ingredient. "
            "4. Cooking Implements Preparation: Enumerate all the cooking tools and implements needed for each phase of the dish's preparation. "
            "Detail any specific preparation these tools might need before the actual cooking starts and describe what pots, pans, dishes, and "
            "other tools will be needed for the final preparation."
            "Remember, your guidance stops at the preparation stage. Do not delve into the actual cooking process of the dish. "
            "Your goal is to set the stage flawlessly for the chef to execute the cooking seamlessly."
            "The recipe you are given is for: {recipe} for {customer_count} people. "
        )

        self.prompt = PromptTemplate(
            input_variables=["recipe", "customer_count"],
            template=template_instruction,
        )

        self.chain = LLMChain(llm=self.llm, prompt=self.prompt)

    def _build_prompt(self, instruction):
        return self.prompt.format(instruction)

    def predict(self, context, model_input, params=None):
        recipe = model_input["recipe"]
        customer_count = model_input["customer_count"]
        run_name = params.get("run_name", "SousChefPreparationRun") if params else "SousChefPreparationRun"

        with mlflow.start_run(run_name=run_name) as run:
            mlflow.log_param("model_name", self.model_name)
            mlflow.log_param("max_tokens", self.max_tokens)
            mlflow.log_param("recipe", recipe)
            mlflow.log_param("customer_count", customer_count)

            try:
                result = self.chain.run({"recipe": recipe, "customer_count": customer_count})

                mlflow.log_metric("preparation_length", len(result))

                with open("preparation_instructions.txt", "w") as f:
                    f.write(result)

                mlflow.log_artifact("preparation_instructions.txt")

                return result

            except Exception as e:
                mlflow.log_param("error", str(e))
                raise e

# Example Usage:
if __name__ == "__main__":
    input_example = {
        "recipe": "Spaghetti Carbonara",
        "customer_count": 4
    }
    signature = mlflow.models.signature.infer_signature(input_example)
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("Cooking Assistant")
    
    with mlflow.start_run(run_name="Cooking_Assistant_Run") as run:
        model_info = mlflow.pyfunc.log_model(
            artifact_path="Cooking_Assistance",
            python_model=SousChefPreparation(),
            pip_requirements=[
                "langchain_groq==0.1.6",  # Specify the correct version if known
                f"langchain=={langchain_version}"
            ],
            input_example=input_example,
            signature=signature,
        )
        
        # Log model details
        mlflow.log_param("langchain_version", langchain_version)
        mlflow.log_param("langchain_groq_version", "0.1.6")
        
        # Print model info for debugging
        print(f"Model logged: {model_info}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Experiment ID: {run.info.experiment_id}")


G:\Anaconda3\Lib\site-packages\mlflow\types\utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/07/07 01:34:15 INFO mlflow.tracking.fluent: Experiment with name 'Cooking Assistant' does not exist. Creating a new experiment.
G:\Anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but im

Model logged: <mlflow.models.model.ModelInfo object at 0x0000022AA36F0310>
Run ID: cb6660e8a28e4796a65380762d3b59f6
Experiment ID: 707330915083176171


In [25]:
def test_model():
    import mlflow.pyfunc

    # Load the model
    model_uri = "runs:/cb6660e8a28e4796a65380762d3b59f6/Cooking_Assistance"  # Replace <run_id> with the actual run ID
    model = mlflow.pyfunc.load_model(model_uri)

    # Example input for prediction
    input_example = {
        "recipe": "Spaghetti Carbonara",
        "customer_count": 4
    }

    # Make a prediction
    result = model.predict(input_example)
    print("Prediction result:", result)

# Replace <run_id> with the actual run ID after registering the model
test_model()

Prediction result: The art of mise-en-place for Spaghetti Carbonara! Let's dive into the meticulous preparation process for this classic Italian dish.

**Ingredients:**

For 4 people:

* 400g spaghetti
* 4 large eggs
* 120g guanciale or pancetta, thinly sliced
* 120g Parmesan cheese, grated
* 20g black pepper, freshly ground
* Salt, to taste
* Fresh parsley, chopped (optional)

**Preparation Techniques:**

1. Spaghetti:
	* Measure and portion out the correct amount of spaghetti.
	* Store in a dry, clean area, away from direct sunlight.
2. Eggs:
	* Crack and separate the eggs into individual containers.
	* Whisk the egg yolks until smooth and set aside.
	* Reserve the egg whites for another use or discard.
3. Guanciale or Pancetta:
	* Slice the guanciale or pancetta into thin strips.
	* Cut the strips into smaller pieces, approximately 1-2 cm in length.
	* Place the pieces on a paper towel-lined plate to drain excess fat.
4. Parmesan Cheese:
	* Grate the Parmesan cheese using a micropla

In [36]:
import mlflow
run_name="Cooking_Assistant_Run"
with mlflow.start_run(run_name=run_name) as run:
    result = mlflow.register_model(
        "runs:/cb6660e8a28e4796a65380762d3b59f6/model",
        "Cooking_Master_V2"
    )

Successfully registered model 'Cooking_Master_V2'.
2024/07/07 02:36:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Cooking_Master_V2, version 1
Created version '1' of model 'Cooking_Master_V2'.


In [37]:
client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name="Cooking_Master_V2",
    version=1,
    stage="Production"
)

C:\Users\mural\AppData\Local\Temp\ipykernel_4732\1113642734.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1720299988301, current_stage='Production', description='', last_updated_timestamp=1720300002492, name='Cooking_Master_V2', run_id='cb6660e8a28e4796a65380762d3b59f6', run_link='', source='mlflow-artifacts:/707330915083176171/cb6660e8a28e4796a65380762d3b59f6/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [35]:
import mlflow.pyfunc

model_name = "Cooking_Master_V1"
stage = 'Production'

model = mlflow.pyfunc.load_model(
    model_uri="runs:/cb6660e8a28e4796a65380762d3b59f6/Cooking_Assistance"
)

input_example = {
        "recipe": "Spaghetti Carbonara",
        "customer_count": 4
    }

result = model.predict(input_example)
print("Prediction result:", result)

Prediction result: The classic Spaghetti Carbonara! Let's get started on the meticulous preparation for this dish.

**Ingredients:**

For 4 people, we'll need:

* 12 oz (340g) spaghetti
* 4 oz (115g) guanciale or pancetta, thinly sliced
* 3 large eggs
* 1 cup (250ml) grated Parmesan cheese
* Salt, black pepper
* Fresh parsley, chopped (optional)
* Extra virgin olive oil

**Preparation Techniques:**

1. **Spaghetti:** No preparation needed, but ensure the spaghetti is stored in an airtight container to maintain freshness.
2. **Guanciale or Pancetta:** Slice the guanciale or pancetta into thin strips, approximately 1/4 inch (6mm) thick. This will help it cook evenly and quickly.
3. **Eggs:** Crack the eggs into a bowl and whisk them lightly with a fork. Set aside.
4. **Parmesan Cheese:** Grate the Parmesan cheese using a microplane or fine grater. This will ensure a smooth, even texture.
5. **Parsley:** Chop the parsley leaves finely, if using as a garnish.

**Ingredient Staging:**

1. *

Serving an MLflow Model from Model Registry

Now run this command from command line

set MLFLOW_TRACKING_URI=http://localhost:5000

make sure to write the different port - other than the one you used while starting mlflow server

mlflow models serve --model-uri models:/Cooking_Master_V1/3 -p 1234 --no-conda  --For new version of mlflow user the version number instead of stage

mlflow models serve --model-uri models:/iris-classifier/Production -p 1234 --no-conda

In [39]:
import requests
import json

# Define the endpoint URL
endpoint = "http://localhost:1234/invocations"

# Define the input example
input_example = {
    "recipe": "Spaghetti Carbonara",
    "customer_count": 4
}

# Convert the input example to the required format
input_data = {"inputs": [input_example]}

# Convert the input data to JSON
input_json = json.dumps(input_data)

# Define the headers
headers = {"Content-Type": "application/json"}

# Send a POST request to the endpoint
response = requests.post(endpoint, data=input_json, headers=headers)

# Print the response
print("Response status code:", response.status_code)
print("Response JSON:", response.json())


Response status code: 200
Response JSON: {'predictions': "The classic Spaghetti Carbonara! Let's break down the mise-en-place process to ensure a flawless execution of this dish.\n\n**Ingredients:**\n\nFor 4 people:\n\n* 12 oz (340g) spaghetti\n* 4 large eggs\n* 4 oz (115g) guanciale or pancetta, sliced into thin strips\n* 1 cup (250ml) grated Parmesan cheese\n* Salt, to taste\n* Freshly ground black pepper, to taste\n* Fresh parsley, chopped (optional)\n\n**Preparation Techniques:**\n\n* Spaghetti: Measure and set aside.\n* Eggs: Crack and whisk together in a bowl, set aside.\n* Guanciale or pancetta: Slice into thin strips, then cut into smaller pieces (about 1 inch / 2.5 cm long). Set aside.\n* Parmesan cheese: Grate and set aside.\n* Salt and black pepper: Measure and set aside.\n* Fresh parsley: Chop and set aside (if using).\n\n**Ingredient Staging:**\n\n* Place the measured spaghetti on a designated station, easily accessible for cooking.\n* Position the whisked eggs near the co